In [32]:
import os
import sys

project_path = os.path.abspath(os.path.join(os.getcwd(),'..','..','..'))
sys.path.append(project_path)
print(f"Project Path: {project_path}")

Project Path: c:\Users\dalej\Documents\_Coding\DragonRegen


In [54]:
from src.MetaFort.SysLogs.KafkaEngine import KafkaEngine
from src.MetaFort.AILoggingTopics import AILoggingTopics
kafka_engine = KafkaEngine('localhost:9092')

In [55]:
# batch_messages = kafka_engine.search_batch_topic(AILoggingTopics.AI_TASK_COMPLETED_TOPIC, search_value='9e7de1ad-c8c4-42af-b3f8-efe55c801b7e', search_key='task_id')
batch_messages = kafka_engine.search_batch_topic(AILoggingTopics.AI_TASK_COMPLETED_TOPIC, search_value='008d1e3e-fde2-4425-9e6c-461a38b0ad19', search_key='task_id')
print("="*30)
print(f"Completed Messages: {batch_messages}")


Searching topic 'ai_tasks_completed' for value: 008d1e3e-fde2-4425-9e6c-461a38b0ad19
This will timeout after 60 seconds if no more messages are available.
Processing 464 messages from partition TopicPartition(topic='ai_tasks_completed', partition=0)
Processing 462 messages from partition TopicPartition(topic='ai_tasks_completed', partition=3)
Found match at offset 461 in partition 3
Processing 492 messages from partition TopicPartition(topic='ai_tasks_completed', partition=4)
Processing 513 messages from partition TopicPartition(topic='ai_tasks_completed', partition=1)
Processing 472 messages from partition TopicPartition(topic='ai_tasks_completed', partition=2)
Search completed in 1.19 seconds.
Processed 2403 messages, found 1 matches.
Completed Messages: [{'topic': 'ai_tasks_completed', 'partition': 3, 'offset': 461, 'timestamp': 1748031635212, 'key': None, 'value': {'task_id': '008d1e3e-fde2-4425-9e6c-461a38b0ad19', 'task_name': 'SchemaRefiner', 'group_task_id': '008d1e3e-fde2-4425-

In [43]:
from kafka import KafkaConsumer
from src.MetaFort.AILoggingTopics import AILoggingTopics

kc = KafkaConsumer(
                    AILoggingTopics.AI_TASK_COMPLETED_TOPIC,
                    bootstrap_servers=['localhost:9092'],
                    group_id=f'task-completed-early-12345',
                    auto_offset_reset='earliest',)

log_list = kc.poll(timeout_ms=1000)

for topic_partition, messages in log_list.items():
    print(f"Topic: {topic_partition.topic}, Partition: {topic_partition.partition}")
    for message in messages:
        print(f"Message: {message.value.decode('utf-8')}")
        # Here you can add your logic to process the message

In [46]:
print(log_list)

{}


In [49]:
log_list = kc.poll(timeout_ms=1000)

for topic_partition, messages in log_list.items():
    print(f"Topic: {topic_partition.topic}, Partition: {topic_partition.partition}")
    for message in messages:
        print(f"Message: {message.value.decode('utf-8')}")
        # Here you can add your logic to process the message
print(log_list)


{}


## Examine Kafka Consumer Group IDs
Look at the current committed offsets of each kafka consumer group_id

In [51]:
group_id = 'task-completed-ef19d41d-a503-42ff-9dcf-6920c21f6b78'

# Initialize the admin client
admin_client = KafkaAdminClient(bootstrap_servers=['localhost:9092'])

# Create a consumer to get end offsets
consumer = KafkaConsumer(bootstrap_servers=['localhost:9092'])

offsets = admin_client.list_consumer_group_offsets(group_id)
    
print(f"\nGroup: {group_id}")
for tp, offset_info in offsets.items():
    print(f"  Topic: {tp.topic}, Partition: {tp.partition}, Offset: {offset_info.offset}")
    
    # Optional: Get lag by comparing with end offsets
    consumer_position = offset_info.offset
    
    # Create a TopicPartition object
    topic_partition = TopicPartition(tp.topic, tp.partition)
    
    # Get the end offset for this topic-partition using the consumer
    consumer.assign([topic_partition])
    end_offset = consumer.end_offsets([topic_partition])[topic_partition]
    
    # Calculate lag
    lag = end_offset - consumer_position
    print(f"    end_offset: {end_offset}")
    print(f"    consumer_position: {consumer_position}")
    print(f"    Lag: {lag}")


Group: task-completed-ef19d41d-a503-42ff-9dcf-6920c21f6b78
  Topic: ai_tasks_completed, Partition: 1, Offset: 139
    end_offset: 139
    consumer_position: 139
    Lag: 0
  Topic: ai_tasks_completed, Partition: 2, Offset: 129
    end_offset: 129
    consumer_position: 129
    Lag: 0
  Topic: ai_tasks_completed, Partition: 4, Offset: 154
    end_offset: 154
    consumer_position: 154
    Lag: 0
  Topic: ai_tasks_completed, Partition: 3, Offset: 127
    end_offset: 127
    consumer_position: 127
    Lag: 0
  Topic: ai_tasks_completed, Partition: 0, Offset: 126
    end_offset: 126
    consumer_position: 126
    Lag: 0


In [48]:
from kafka import KafkaAdminClient
from kafka.admin import NewTopic
from kafka import KafkaConsumer
from kafka import TopicPartition

# Initialize the admin client
admin_client = KafkaAdminClient(bootstrap_servers=['localhost:9092'])

# List all consumer groups
consumer_groups = admin_client.list_consumer_groups()
group_ids = [group[0] for group in consumer_groups]  # Extract group IDs from tuples
print(f"Available consumer groups: {group_ids}")

# Create a consumer to get end offsets
consumer = KafkaConsumer(bootstrap_servers=['localhost:9092'])

# Get offsets for each consumer group
for group_id in group_ids:
    # Get offsets for all topics this consumer group is consuming
    offsets = admin_client.list_consumer_group_offsets(group_id)
    
    print(f"\nGroup: {group_id}")
    for tp, offset_info in offsets.items():
        print(f"  Topic: {tp.topic}, Partition: {tp.partition}, Offset: {offset_info.offset}")
        
        # Optional: Get lag by comparing with end offsets
        consumer_position = offset_info.offset
        
        # Create a TopicPartition object
        topic_partition = TopicPartition(tp.topic, tp.partition)
        
        # Get the end offset for this topic-partition using the consumer
        consumer.assign([topic_partition])
        end_offset = consumer.end_offsets([topic_partition])[topic_partition]
        
        # Calculate lag
        lag = end_offset - consumer_position
        print(f"    end_offset: {end_offset}")
        print(f"    consumer_position: {consumer_position}")
        print(f"    Lag: {lag}")

Available consumer groups: ['task-completed-early-12345', 'task-completed_early-1234', 'default', 'task-completed-1234', 'action-agent', 'column_sub']

Group: task-completed-early-12345
  Topic: ai_tasks_completed, Partition: 1, Offset: 0
    end_offset: 64
    consumer_position: 0
    Lag: 64
  Topic: ai_tasks_completed, Partition: 2, Offset: 0
    end_offset: 62
    consumer_position: 0
    Lag: 62
  Topic: ai_tasks_completed, Partition: 4, Offset: 0
    end_offset: 70
    consumer_position: 0
    Lag: 70
  Topic: ai_tasks_completed, Partition: 3, Offset: 0
    end_offset: 62
    consumer_position: 0
    Lag: 62
  Topic: ai_tasks_completed, Partition: 0, Offset: 0
    end_offset: 63
    consumer_position: 0
    Lag: 63

Group: task-completed_early-1234

Group: default
  Topic: ai_tasks_completed, Partition: 1, Offset: 64
    end_offset: 64
    consumer_position: 64
    Lag: 0
  Topic: ai_tasks_completed, Partition: 2, Offset: 62
    end_offset: 62
    consumer_position: 62
    Lag: 0

In [20]:
# Initialize the admin client
admin_client = KafkaAdminClient(bootstrap_servers=['localhost:9092'])

admin_client.delete_topics(['ai_tasks_completed'])
print("Topic 'ai_tasks_log' deleted successfully.")

Topic 'ai_tasks_log' deleted successfully.


## Look at all topics

In [24]:
from kafka import KafkaAdminClient
from kafka.admin import NewTopic
from kafka import KafkaConsumer
from kafka import TopicPartition

# Initialize the admin client
admin_client = KafkaAdminClient(bootstrap_servers=['localhost:9092'])

# List all consumer groups
consumer_groups = admin_client.list_consumer_groups()
print(f"Available consumer groups: {consumer_groups}")

# List all topics
topics = admin_client.list_topics()
print(f"Available topics: {topics}")

# Find the length of ai_tasks_completed topic

topic_name = 'ai_tasks_completed'
if topic_name in topics:
    print(admin_client.describe_topics([topic_name])[0])
    # max offset of this topic
    topic_partition = TopicPartition(topic_name, 0)
    consumer = KafkaConsumer(bootstrap_servers=['localhost:9092'])
    consumer.assign([topic_partition])
    end_offset = consumer.end_offsets([topic_partition])[topic_partition]
    print(f"End offset for topic '{topic_name}': {end_offset}")

Available consumer groups: [('action-agent', 'consumer'), ('column_sub', 'consumer')]
Available topics: ['ai_tasks_completed', 'quickstart-events', 'ai_requests_log', 'ai_tasks_log', 'ai_tasks', 'many_tasks', '__consumer_offsets']
{'error_code': 0, 'topic': 'ai_tasks_completed', 'is_internal': False, 'partitions': [{'error_code': 0, 'partition': 1, 'leader': 1, 'replicas': [1], 'isr': [1], 'offline_replicas': []}, {'error_code': 0, 'partition': 3, 'leader': 1, 'replicas': [1], 'isr': [1], 'offline_replicas': []}, {'error_code': 0, 'partition': 2, 'leader': 1, 'replicas': [1], 'isr': [1], 'offline_replicas': []}, {'error_code': 0, 'partition': 0, 'leader': 1, 'replicas': [1], 'isr': [1], 'offline_replicas': []}, {'error_code': 0, 'partition': 4, 'leader': 1, 'replicas': [1], 'isr': [1], 'offline_replicas': []}]}
End offset for topic 'ai_tasks_completed': 4


In [14]:
from kafka.admin import KafkaAdminClient, NewTopic
admin_client = KafkaAdminClient(bootstrap_servers=['localhost:9092'])

new_topic = NewTopic(
    name='many_tasks',
    num_partitions=5,
    replication_factor=1
)
admin_client.create_topics([new_topic])


CreateTopicsResponse_v3(throttle_time_ms=0, topic_errors=[(topic='many_tasks', error_code=0, error_message=None)])

In [22]:
from kafka.admin import KafkaAdminClient, ConfigResource, ConfigResourceType
from kafka import KafkaConsumer

# Initialize the admin client
admin_client = KafkaAdminClient(bootstrap_servers=['localhost:9092'])

# Method 1: Get all topics and their partitions using describe_topics() 
topic_metadata = admin_client.describe_topics()
for topic_info in topic_metadata:
    print(topic_info)
    partition_count = len(topic_info['partitions'])
    print(f"Topic: {topic_name}, Partitions: {partition_count}")

{'error_code': 0, 'topic': 'ai_tasks_completed', 'is_internal': False, 'partitions': [{'error_code': 0, 'partition': 1, 'leader': 1, 'replicas': [1], 'isr': [1], 'offline_replicas': []}, {'error_code': 0, 'partition': 3, 'leader': 1, 'replicas': [1], 'isr': [1], 'offline_replicas': []}, {'error_code': 0, 'partition': 2, 'leader': 1, 'replicas': [1], 'isr': [1], 'offline_replicas': []}, {'error_code': 0, 'partition': 0, 'leader': 1, 'replicas': [1], 'isr': [1], 'offline_replicas': []}, {'error_code': 0, 'partition': 4, 'leader': 1, 'replicas': [1], 'isr': [1], 'offline_replicas': []}]}
Topic: ai_tasks_completed, Partitions: 5
{'error_code': 0, 'topic': 'quickstart-events', 'is_internal': False, 'partitions': [{'error_code': 0, 'partition': 0, 'leader': 1, 'replicas': [1], 'isr': [1], 'offline_replicas': []}]}
Topic: ai_tasks_completed, Partitions: 1
{'error_code': 0, 'topic': 'ai_requests_log', 'is_internal': False, 'partitions': [{'error_code': 0, 'partition': 1, 'leader': 1, 'replicas